In [1]:
import collections
import json
import keras
import numpy as np
import os
import sys
sys.path.append("../../../ecg")
import scipy.stats as sst

import util
import load

Using TensorFlow backend.


In [4]:
#model_path = "/home/q/Documents/master/TEL531E/termproject/ecg-master/saved/cinc17/1607813643-299/0.404-0.859-008-0.364-0.870.hdf5"
#data_path = "/home/q/Documents/master/TEL531E/termproject/ecg-master/examples/cinc17/train.json"

#model_path = "/home/q/Documents/master/TEL531E/termproject/ecg-master/saved/cinc17/1607813643-299/0.404-0.859-008-0.364-0.870.hdf5"
#data_path = "/home/q/Documents/master/TEL531E/termproject/ecg-master/examples/cinc17/val.json"

data = load.load_dataset(data_path)
preproc = util.load(os.path.dirname(model_path))
model = keras.models.load_model(model_path)

100%|██████████| 270/270 [00:00<00:00, 2133.05it/s]


In [5]:
data_path = "/home/q/Documents/master/TEL531E/termproject/ecg-master/examples/cinc17/train.json"
with open(data_path, 'r') as fid:
    # get each data labels
    train_labels = [json.loads(l)['labels'] for l in fid]
# count values of the each label
counts = collections.Counter(preproc.class_to_int[l[0]] for l in train_labels)
counts = sorted(counts.most_common(), key=lambda x: x[0])
counts = zip(*counts)[1]
smooth = 500
counts = np.array(counts)[None, None, :]
total = np.sum(counts) + counts.shape[1]
prior = (counts + smooth) / float(total)

In [6]:
probs = []
labels = []
for x, y  in zip(*data):
    x, y = preproc.process([x], [y])
    probs.append(model.predict(x))
    labels.append(y)

In [7]:
preds = []
ground_truth = []
for p, g in zip(probs, labels):
    preds.append(sst.mode(np.argmax(p / prior, axis=2).squeeze())[0][0])
    ground_truth.append(sst.mode(np.argmax(g, axis=2).squeeze())[0][0])

In [8]:
import sklearn.metrics as skm
report = skm.classification_report(
            ground_truth, preds,
            target_names=preproc.classes,
            digits=3)
scores = skm.precision_recall_fscore_support(
                    ground_truth,
                    preds,
                    average=None)
print(report)
print "CINC Average {:3f}".format(np.mean(scores[2][:3]))

              precision    recall  f1-score   support

           A      0.953     0.872     0.911        47
           N      0.934     0.955     0.944       133
           O      0.860     0.778     0.817        63
           ~      0.794     1.000     0.885        27

   micro avg      0.904     0.904     0.904       270
   macro avg      0.885     0.901     0.889       270
weighted avg      0.906     0.904     0.903       270

CINC Average 0.890672
